In [1]:
# prompt: テキストファイルを読み込んで整形する

import re
import glob

def format_text_file(file_path):

  #変数の初期化
  bits = []
  four_byte = []

  #テキストファイルを開く
  with open(file_path, 'r') as f:

    while True:

        bits = []
        text = f.readline()

        if not text:
          break
        else:
          #バイト列の取得
          text = text[9:32]+text[34:58]
          text = re.sub(r'[\n\r]', '', text)

          for i in text.split():
            bits.append(i)

          #リトルエンディアンの変換

          for i in range(0,len(bits),2):
            four_byte.append(bits[i+1]+bits[i])

    f.close()
    return four_byte

def hex_to_signed_int(hex_str, bits=16):
    # 16進数文字列を10進数の整数に変換
    unsigned_int = int(hex_str, 16)

    # 符号ビットを考慮して符号付き整数に変換
    if unsigned_int >= 2**(bits - 1):
        signed_int = unsigned_int - 2**bits
    else:
        signed_int = unsigned_int

    return signed_int

#テキストファイルの総取得
data = glob.glob('/content/*.txt')

#出力は配列から除く
if "/content/output.txt" in data:
  data.remove("/content/output.txt")

#0以下または2つ以上ファイルがあるときの例外処理，
if len(data) == 0:
  print("No text files found.")
  exit()

if len(data) > 1:
  print("More than one text file found.")
  exit()

#処理の実行
hexs = format_text_file(data[0])
print(hexs)

print()

#出力
f = open('/content/output.txt',"w")
print(*list(map(lambda x:hex_to_signed_int(x,16),hexs)),file=f)
f.close()




ValueError: list.remove(x): x not in list